# Generate Splits

In [ ]:
import os

import pandas as pd

In [ ]:
username = ""
source = ""
media = ""

In [ ]:
data_path = os.path.join("../../data/recommendations", username)

In [ ]:
def fullpath(x):
    return os.path.join(data_path, f"user_{x}_list.csv")

In [ ]:
df = pd.read_csv(fullpath(media))
df = df.sort_values(by=["update_order", "updated_at"]).reset_index(drop=True)
df["unit"] = 1
df["order"] = df.groupby("userid")["unit"].apply(lambda x: x.cumsum()[::-1]).values
df.to_csv(fullpath(media), index=False)